In [1]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

import datetime

from data.generator import generateDataSetCat

2023-01-04 15:05:57.215086: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
def evaluate_model(model, in_data, out_data):
    loss, acc = model.evaluate(in_data, out_data)
    pred_data = model.predict(in_data)
    
    faults = np.sum(pred_data != out_data)
    
    mean_cat_diff = np.sum(pred_data - out_data)/faults
    abs_cat_diff = np.sum(np.abs(pred_data - out_data))/faults
    
    return loss, acc, mean_cat_diff, abs_cat_diff, faults

In [6]:
courses = [
    {
        "code": "WT",
        "start": datetime.datetime(2020, 9, 15, 0, 0).timestamp(),
        "stop": datetime.datetime(2021, 2, 10, 0, 0).timestamp(),
    },
    {
        "code": "WT_2",
        "start": datetime.datetime(2020, 9, 15, 0, 0).timestamp(),
        "stop": datetime.datetime(2021, 2, 10, 0, 0).timestamp(),
    },
    {
        "code": "DN",
        "start": datetime.datetime(2020, 9, 15, 0, 0).timestamp(),
        "stop": datetime.datetime(2021, 2, 10, 0, 0).timestamp(),
    },
    {
        "code": "RS",
        "start": datetime.datetime(2021, 2, 1, 0, 0).timestamp(),
        "stop": datetime.datetime(2021, 6, 30, 0, 0).timestamp(),
    },
    {
        "code": "KW1",
        "start": datetime.datetime(2021, 2, 1, 0, 0).timestamp(),
        "stop": datetime.datetime(2021, 6, 30, 0, 0).timestamp(),
    },
    {
        "code": "KW2",
        "start": datetime.datetime(2020, 9, 15, 0, 0).timestamp(),
        "stop": datetime.datetime(2021, 2, 10, 0, 0).timestamp(),
    },
]

for course in courses:
    course_code = course["code"]
    print(f"=== {course_code} ===")
    in_data, out_data = generateDataSetCat("./datasets/raw/logs_" + course_code + ".csv", 
                            "./datasets/raw/Resultaten_" + course_code + ".csv", 
                            course["start"], 
                            course["stop"],
                            100)
    
    in_data = np.array(in_data)
    out_data = np.array(out_data)
    
    print("== With Zero ==")
    model = tf.keras.models.load_model("models/var_time/" + course_code + ".h5")
    
    print(evaluate_model(model, in_data, out_data))    
    
    print("== No Zero ==")
    model = tf.keras.models.load_model("models/var_time_no_zero/" + course_code + ".h5")
    print(evaluate_model(model, in_data, out_data))
          
    print()

=== WT ===
== With Zero ==
529/529 [==============================] - 1s 1ms/step
(0.20005935430526733, 0.9188757538795471, -1.2334079639446256, 1.273388773254429, 50024)
== No Zero ==
529/529 [==============================] - 1s 1ms/step
(0.786058783531189, 0.9090532660484314, -1.231758199763966, 1.2716855320483924, 50091)

=== WT_2 ===
== With Zero ==
1022/1022 [==============================] - 1s 1ms/step
(0.6197062134742737, 0.6648623943328857, -0.9757770364113804, 1.1004769772221286, 96231)
== No Zero ==
1022/1022 [==============================] - 1s 1ms/step
(1.1818948984146118, 0.6395106911659241, -0.9683407238536329, 1.0920903371606048, 96970)

=== DN ===
== With Zero ==
1060/1060 [==============================] - 1s 1ms/step
(0.6346023082733154, 0.6530088782310486, -0.7995225512666092, 0.9909140490655091, 97183)
== No Zero ==
1060/1060 [==============================] - 2s 1ms/step
(0.8213109374046326, 0.6429203748703003, -0.7946816669996096, 0.9849143442202164, 97775)

==